In [1]:
"""Take dreamcoder models from diff expts, (but overlapping behavior), and combine human/model scores
Also get various controls (e.g, random parses)
"""

# %load_ext autoreload
# %autoreload 2

from analysis.getModelHumanDists import * 
from analysis.modelAnalyses import *


HERE


In [ ]:
################################# - NEW VERSION, ONE FUNCITON TO LOAD

In [ ]:
################################## OLD VERSION - LOADING EACH COMPONENT

In [ ]:
# ===== load multiple models and concatenate
from pythonlib.tools.dicttools import printOverviewKeyValues
from analysis.modelAnalyses import *

ECTRAINlist = ["S12.10.test4", "S13.10.test4"]
modelkind_list = ["parse", "randomperm"]
ver="aggregate"
use_withplannerscore=True
distances_flat, DAT_all, workerlist, SAVEDIR = loadMultDCHumanDistances(ECTRAINlist, modelkind_list, ver, use_withplannerscore)

In [ ]:
# DAT_all, workerlist, SAVEDIR = loadMultDCdata(ECTRAINlist)
addWorkerCond(distances_flat, workerlist)
print("Extracted {} workers".format(len(set([d["workerID"] for d in workerlist]))))

In [ ]:
from analysis.importDrawgood import dgplan as D
Planner = D.Planner

BATCHNAME = "191108"
BATCHNAME = "191116"
EXPT = "2.4"

Planner, summarydict_all, datall, savedir = loadPlannerData(EXPT, BATCHNAME)

In [ ]:
params = D.getParamsAggregate(D.flattenSummary(summarydict_all), returnnames=False)
params

In [ ]:
# 1) to df
import pandas as pd
from pythonlib.tools.pandastools import aggregGeneral

df = pd.DataFrame(distances_flat)

In [ ]:
# -- summarize data
printOverviewKeyValues(distances_flat)

In [ ]:

# === plotting all data in violin plot
import seaborn as sns
from modelPlanning import addLabel
import matplotlib.pyplot as plt

for stim in set(df["stim"].values):
    dfthis = df[df["stim"]==stim]
#     fig, ax = plt.subplots()
#     fig.set_size_inches(11, 8)

    ax = sns.catplot(data=dfthis, hue="model", x="human", y="dist", kind="violin", height=8, aspect=25/8)
    addLabel(ax)
    ax.savefig("{}/overview_eachstim_{}.pdf".format(SAVEDIR, stim))


In [ ]:
## AGGREGATING OVER PARSES, USING PLANNER COSTS
## OBSOLETE - TOO SLOW
plannermodel="motor"

stimlist = [d["stim"] for d in distances_flat]
humanlist = [d["human"] for d in distances_flat]
modellist = [d["model"] for d in distances_flat]

for stim in stimlist:
    for human in humanlist:
        for model in modellist:
            # -- get all datapoints
            dists = filterDistances(distances_flat, [stim], [human], [model])
            
            print("SUM: shouldnt this be 1?")
            print(np.sum([d["{}_prob".format(plannermodel)] for d in dists]))
            print("reweighting all distances using planner model scores")
            
            for d in dists:
                prob = d["{}_prob".format(plannermodel)]
                d["dist"] = d["dist"] * prob                
            
            
            

In [ ]:
for i, d in enumerate(distances_flat):
    print(i)
    prob = d["{}_prob".format(plannermodel)]
    d["dist"] = d["dist"] * prob                


In [ ]:
SD = "{}/summarydict_planner_{}.pickle".format(SAVEDIR, plannermodel)

allsummarydict = {
	"DAT_all":DAT_all,
	"distances_flat":distances_flat,
	"planner_summarydict_all":summarydict_all,
	"planner_params":params
}
with open(SD, "wb") as f:
	pickle.dump(allsummarydict, f)

In [ ]:
## AGGREGATE OVER PARSES, using percentile in lambda function
PRCTILE=25

import numpy as np
am = lambda x: np.percentile(x, PRCTILE)
df = aggregGeneral(df, group=["stim", "human", "model"], values=["dist"], nonnumercols=["human_cond"],
              aggmethod=[am])
# df = aggreg(df, group=["stim", "human", "model"], values=["dist"], aggmethod=[am])
# df["dist"]=df["dist_<lambda>"]
# df = df.drop(columns=["dist_<lambda>"])

In [ ]:
# import sys
# def sizeof_fmt(num, suffix='B'):
#     ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
#     for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
#         if abs(num) < 1024.0:
#             return "%3.1f %s%s" % (num, unit, suffix)
#         num /= 1024.0
#     return "%.1f %s%s" % (num, 'Yi', suffix)

# for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
#                          key= lambda x: -x[1])[:10]:
#     print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))


In [ ]:
# PLOT EACH STIM [ONE SUBPLOT EACH STIM, SHOWING THE POINT ESTIMATE FOR EACH SUBECT]
import seaborn as sns

ax = sns.catplot(data=df, hue="model", x="human", y="dist", col="stim", col_wrap=6)
from modelPlanning import addLabel
addLabel(ax)
ax.savefig("{}/overview.pdf".format(SAVEDIR))

In [ ]:
## WAYS TO FILTER DF
# - get stims that solved by all models
def filterOnlyStimWithAllModels(df, behexpt="2.4", MODELS = 
                                ["S12.10", "S13.10.test4", "S12.10_randomperm", "S13.10.test4_randomperm"]):
    """given a dataframe, filters so that only only keeps stim that have dat for all models.
    Assumes that only expect one datapoint in df per model (i.e, need to aggregate first)"""
    from analysis.importDrawgood import dgexpt
    
    teststims = dgexpt.getTestStim(behexpt, removepng=True)
    
    teststims_good =[]
    for stim in teststims:
        successful_models = set(df[df["stim"]==stim]["model"])
        if len([s for s in successful_models if s in MODELS])==len(MODELS):
            teststims_good.append(stim)   
    
    return df[df["stim"].isin(teststims_good)].reset_index()


## WAYS TO MODIFY/AGGREGATE
def aggregOverStim(df):
    """averages over all stimuli for a given human/model
    note: will not check for you whether data are balanced"""
    return aggregGeneral(df, group=["human", "model", "human_cond"], values=["dist"], aggmethod=["mean"])

## THINGS TO APPLY TO df_human
def applyFunctionToGroups(df_human, groups=["human", "human_cond"], model1="S12.10.test4", model2="S13.10.test4"):
    """df_human already has aggreageted over stims, so that each human is one dat"""
    F = lambda x: x.loc[x["model"]==model2]["dist"].values[0] - x.loc[x["model"]==model1]["dist"].values[0]  # note, the [0] is correct, since only one value and is in a list
    Fname = model2 + "-" + model1
    df_human = df_human.groupby(groups).apply(F).reset_index(name=Fname)
    return df_human



In [ ]:
import seaborn as sns
from modelPlanning import addLabel

MODEL_LIST = ["S12.10.test4", "S13.10.test4", "S12.10.test4_randomperm", "S13.10.test4_randomperm"]
# MODEL_LIST = ["S12.10", "S13.10.test4", "S12.10_randomperm", "S13.10.test4_randomperm"]



In [ ]:
# ================ COMPARE DC MODEL TO PLANNER MODEL
dfthis = df_weighted

################# 1) preprocess
dfthis = filterOnlyStimWithAllModels(dfthis, behexpt=DAT_all[0]["behaviorexpt"], MODELS = MODEL_LIST)
dfthis

In [ ]:
# take mean over stimuli
dfworkers = aggregOverStim(dfthis)
dfworkers

In [ ]:
# === plot without subtracting modls
ax = sns.catplot(data=dfworkers, hue="model", x="human_cond", y="dist", dodge=True)
sns.catplot(data=dfworkers, hue="model", x="human_cond", y="dist", dodge=True, kind="boxen")
ax.savefig("{}/teststimcommon_summarize.pdf".format(SAVEDIR))

# ===
plt.figure(figsize=(20,10))
ax = sns.catplot(data=dfworkers, x = "human", row="human_cond", y="dist", hue="model", height=3, aspect=15/3)
addLabel(ax)
ax.savefig("{}/teststimcommon_overview.pdf".format(SAVEDIR))

In [ ]:
################# 2) for each worker get a diferences between moedls.
model2 = "S13.10.test4"
model1 = "S12.10.test4"
dfworkers = applyFunctionToGroups(dfworkers, model1=model1, model2=model2)

In [ ]:
colname = model2 + "-" + model1
params = planner_params

In [ ]:
# check that set of workers are identical from two experimetns
assert set(dfworkers["human"]) == set([col for col in params.columns if col not in ["xname", "xnum"]]), "names of workers do not perfeclty match between drawgood and dreamcoder"

# merge workers and params
dfworkers = pd.merge(dfworkers, params.transpose().reset_index().rename(columns = {'workerID':'human'}), on="human")
dfworkers = dfworkers.rename(columns={i:"x{}".format(i) for i in range(6)})

# -- plot
plt.figure(figsize=(20,30))
ax = plt.subplot(3,2,1)
ax = sns.regplot(x=colname, y="x4", ci=95, data=dfworkers)
ax = sns.scatterplot(x=colname, y="x4", hue="human_cond", data=dfworkers)

ax = plt.subplot(3,2,2)
ax = sns.scatterplot(x="x4", y="x5", hue="human_cond", data=dfworkers)

ax = plt.subplot(3,2,3)
sns.scatterplot(x=colname, y="x4", hue="x5", data=dfworkers, ax=ax)

ax = plt.subplot(3,2,4)
sns.scatterplot(x=colname, y="x4", hue="x3", data=dfworkers, ax=ax)

ax = plt.subplot(3,2,5)
sns.stripplot(data=dfworkers, x="human_cond", y=colname, color='k')
sns.boxplot(data=dfworkers, x="human_cond", y=colname)

ax = plt.subplot(3,2,6)
sns.stripplot(data=dfworkers, x="human_cond", y="x4", color='k')
sns.boxplot(data=dfworkers, x="human_cond", y="x4")

plt.savefig("{}/teststimcommon_vsplanner.pdf".format(SAVEDIR))

In [ ]:
# === for each model get comparison to its own controls
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant

X = dfworkers[colname].values.reshape(-1,1)
y = dfworkers["x4"].values.reshape(-1,1)
X = add_constant(X, prepend=False)

res = OLS(y, X).fit()
res.summary()


X = dfworkers[colname].values.reshape(-1,1)
y = dfworkers["human_cond"].values.reshape(-1,1)
X = add_constant(X, prepend=False)

res = OLS(y, X).fit()
res.summary()


In [ ]:
df

In [ ]:
dfthis = df_weighted

# ===== AVERAGING OVER SUBJECT (EEACH STIM A DATAPOINT)
# SUMMARY PLOT - group across subjects that were in same condition
dfthis = aggregGeneral(dfthis, group=["stim", "model", "human_cond"], values=["dist"], aggmethod=["mean"])

# -- only test stims
# - get list of test stimuli
from analysis.importDrawgood import dgexpt
teststims = dgexpt.getTestStim(DAT_all[0]["behaviorexpt"], removepng=True)

stimlist = teststims
dfthis = dfthis[dfthis["stim"].isin(stimlist)]
ax = sns.catplot(data=dfthis, hue="model", x="human_cond", y="dist", dodge=True)
sns.catplot(data=dfthis, hue="model", x="human_cond", y="dist", kind="boxen", dodge=True)
addLabel(ax)

# -- one plot for each stim
stimlist = teststims
dfthis = dfthis[dfthis["stim"].isin(stimlist)]
ax = sns.catplot(data=dfthis, hue="model", x="human_cond", col="stim", col_wrap=6, y="dist")
addLabel(ax)


# -- only test stims, and only if has all moedls
# stimlist = teststims
# dfthis = dfthis[dfthis["stim"].isin(stimlist)]
# dfthis = dfthis[dfthis.groupby(["stim"]).agg(lambda x: len(x))==8]
SC = dfthis[["dist", "stim"]].groupby("stim").agg(lambda x: len(x))
dfthis = dfthis.merge(SC, left_on = 'stim', right_index=True)
df_allmodels = dfthis[dfthis["dist_y"]==8.0]

ax = sns.catplot(data=df_allmodels, hue="model", x="human_cond", y="dist_x", dodge=True)
sns.catplot(data=df_allmodels, hue="model", x="human_cond", y="dist_x", kind="boxen")

addLabel(ax)

In [ ]:

# EACH DATAPOINT 
# -- only test stims, and only if has all moedls
stimlist = teststims
dfthis = dfthis[dfthis["stim"].isin(stimlist)]
dfthis = dfthis[dfthis.groupby(["stim"]).agg(lambda x: len(x))==8]

ax = sns.catplot(data=df_allmodels, hue="model", x="human_cond", y="dist_x",dodge=True, kind="boxen")
addLabel(ax)